### Advantage Actor-Critic (A2C)

**Principle:** Dual Critic-Actor Learning

**Definition:** Combines policy gradient (actor) with learned value function (critic) to reduce variance (A2C).

**Algorithm Description:** Advantage Actor-Critic uses two neural networks: an actor that learns the policy and a critic that learns a value function to estimate advantages (how much better an action is than average). The critic's advantage estimates reduce variance in policy gradient updates, stabilizing and accelerating learning.

**Typical Use Cases:**
- Combines policy-based (actor) and value-based (critic) methods
- Synchronous version of a3c
- Variance reduction in policy gradients
- Works with continuous and discrete action spaces

**Assumptions:**
- Continuous/discrete actions
- Large datasets
- On-policy learning
- Variance reduction

### ⚙️ Prerequisites & Setup

**Required Packages:**
- `gymnasium>=0.29.0` - RL environment
- `torch>=2.0.0` - Deep learning framework
- `numpy`, `matplotlib`, `seaborn` - Data processing and visualization

**Installation:**
```bash
pip install gymnasium torch numpy matplotlib seaborn
```

**⏱️ Estimated Training Time:**
- CPU: ~10-15 minutes
- GPU: ~4-6 minutes

**💡 Quick Test:** To reduce training time, change `n_episodes=500` to `n_episodes=100` in the training cell.

### 1. Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
torch.manual_seed(42)

### 2. A2C (Advantage Actor-Critic)

A2C combines value-based and policy-based methods:

**Actor:** Updates policy using advantage
**Critic:** Estimates state value V(s)

**Advantage:** A(s,a) = Q(s,a) - V(s) = r + γV(s') - V(s)

**Actor Loss:** -log π(a|s) * A(s,a)
**Critic Loss:** (V(s) - target)²

In [ ]:
class A2CNetwork(nn.Module):
    """Shared network for actor and critic"""
    
    def __init__(self, state_dim, action_dim, hidden=128):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(state_dim, hidden),
            nn.ReLU()
        )
        self.actor = nn.Linear(hidden, action_dim)
        self.critic = nn.Linear(hidden, 1)
    
    def forward(self, state):
        x = self.shared(state)
        return F.softmax(self.actor(x), dim=-1), self.critic(x)

In [ ]:
class A2CAgent:
    def __init__(self, state_dim, action_dim, lr=3e-4, gamma=0.99):
        self.gamma = gamma
        self.network = A2CNetwork(state_dim, action_dim)
        self.optimizer = optim.Adam(self.network.parameters(), lr=lr)
    
    def select_action(self, state):
        state = torch.FloatTensor(state)
        with torch.no_grad():
            probs, _ = self.network(state)
            dist = Categorical(probs)
            action = dist.sample()
        return action.item()
    
    def update(self, states, actions, rewards, next_states, dones):
        states = torch.FloatTensor(states)
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(next_states)
        dones = torch.FloatTensor(dones)
        
        # Current policy and values
        probs, values = self.network(states)
        dist = Categorical(probs)
        log_probs = dist.log_prob(actions)
        
        # Compute targets and advantages
        with torch.no_grad():
            _, next_values = self.network(next_states)
            targets = rewards + self.gamma * next_values.squeeze() * (1 - dones)
            advantages = targets - values.squeeze()
        
        # Actor loss (policy gradient)
        actor_loss = -(log_probs * advantages).mean()
        
        # Critic loss (value function)
        critic_loss = F.mse_loss(values.squeeze(), targets)
        
        # Entropy bonus for exploration
        entropy = dist.entropy().mean()
        
        # Total loss
        loss = actor_loss + 0.5 * critic_loss - 0.01 * entropy
        
        self.optimizer.zero_grad()
        loss.backward()
        
        # Gradient clipping for training stability
        torch.nn.utils.clip_grad_norm_(self.network.parameters(), max_norm=0.5)
        
        self.optimizer.step()
        
        return loss.item()

### 3. Train A2C Agent

In [ ]:
env = gym.make('CartPole-v1')
agent = A2CAgent(env.observation_space.shape[0], env.action_space.n)

episode_rewards = []
batch_size = 5

states_batch, actions_batch, rewards_batch = [], [], []
next_states_batch, dones_batch = [], []

print('Training A2C...')
for episode in range(500):
    state, _ = env.reset()
    total_reward = 0
    
    for t in range(500):
        action = agent.select_action(state)
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        
        states_batch.append(state)
        actions_batch.append(action)
        rewards_batch.append(reward)
        next_states_batch.append(next_state)
        dones_batch.append(done)
        
        total_reward += reward
        state = next_state
        
        # Update every batch_size steps
        if len(states_batch) >= batch_size:
            agent.update(states_batch, actions_batch, rewards_batch, 
                        next_states_batch, dones_batch)
            states_batch, actions_batch, rewards_batch = [], [], []
            next_states_batch, dones_batch = [], []
        
        if done:
            break
    
    episode_rewards.append(total_reward)
    
    if (episode + 1) % 50 == 0:
        print(f'Episode {episode+1}, Avg Reward: {np.mean(episode_rewards[-50:]):.2f}')

### 4. Visualize Results

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(episode_rewards, alpha=0.3, label='Raw')
if len(episode_rewards) > 20:
    ma = np.convolve(episode_rewards, np.ones(20)/20, mode='valid')
    plt.plot(ma, linewidth=2, label='MA(20)')
plt.axhline(195, color='r', linestyle='--', label='Solved')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('A2C Training Performance')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print(f'\nFinal Performance: {np.mean(episode_rewards[-100:]):.2f}')
env.close()